## Trabalhando com Elasticsearch
<hr>
<br>
by: Thiago Ribeiro, Diogo Tallys e José Endson

Crie uma base de dados no SGBD escolhido na atividade anterior contendo um esquema simples e realize as seguintes ações.

- Inserir ao menos 4 novas instâncias;

- Atualizar 2 instâncias;

- Realizar 3 consultas diferentes;

- Remover 1 instância.

In [3]:
from elasticsearch import Elasticsearch
import pandas as pd
from tqdm import tqdm

#### Conectando e criando novo _index_ no Elasticsearch
<hr>
<br>
    Os primeiros passos envolvem iniciar uma conexão com a api do Elasticsearch que está rodando localmente no PC. A partir do link do servidor, do certificado e as informações de autenticação, a conexão é iniciada.
<br>
<br>
    Em seguida, a base é carregada em um dataframe do Pandas e então tratada para evitar erros na inserção dos dados.
<br>
<br>
    É feito, então, o mapping (schema) do novo index que será criado. Nele, são passadas as configurações, que neste caso serão apenas as informações dos tipos aceitos em cada campo de uma instância. É então criado o index com os métodos do cliente Elasticsearch

```python 
client.indices.create(index = ..., body = ...)
```

In [2]:
# CRIANDO UMA CONEXÃO COM O ELASTICSEARCH QUE ESTÁ RODANDO LOCALMENTE
with open(".secret", "r") as f:
    SECRET = f.read().strip()

client = Elasticsearch("https://localhost:9200",
                       ca_certs = "http_ca.crt",
                       basic_auth = ("elastic", str(SECRET)))

if client.ping():
    print("Conexão realizada!")
    print(client.info())
else:
    print("Erro de conexão...")

Conexão realizada!
{'name': 'DESKTOP-I132PNT', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'PUJYnly-ThWt6Zx1vDvPcw', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [8]:
# BASE DE DADOS QUE SERÁ COLOCADA NO INDEX DO ES
dt = pd.read_csv("games.csv").iloc[:, 1:]
dt['release_date'] = pd.to_datetime(dt['release_date'])
dt['release_date'] = dt['release_date'].dt.strftime('%d/%m/%Y')
dt = dt.fillna(" ")
print(dt.shape)
dt = dt.sample(100)
print(f"Sample shape: {dt.shape}")
dt.head()

(20022, 7)
Sample shape: (100, 7)


,metascore,platform,release_date,sort_no,summary,title,user_score
8054,75,DS,06/05/2008,8055,Players take on the role of the characters fro...,Speed Racer,7.4
19455,43,DS,04/12/2007,19456,The Golden Compass follows the screenplay adap...,The Golden Compass,5.2
1331,86,Switch,14/11/2017,1332,Rocket League is a high-powered hybrid of arca...,Rocket League,7.8
427,92,PC,30/11/1998,253,Sneak through the shadows of 12 treacherous mi...,Thief: The Dark Project,8.9
2612,83,Xbox One,15/05/2018,2613,Imagine a place where everything that is lost ...,Forgotton Anne,7.4


client.indices.delete(index = 'metacritic_games')

In [27]:
# CRIANDO UM NOVO INDEX

# mapping (Schema) dos dados do novo index
index_config = {
    'mappings': {
        'properties':{
            'metascore' : { 'type' : 'long' },
            'platform' : { 'type' : 'text' },
            'release_date' : { 'type' : 'date' },
            'sort_no' :  { 'type' : 'long' },
            'summary' :  { 'type' : 'text' },
            'title' :  { 'type' : 'text' },
            'user_score': { 'type' : 'text' }
        }
    }
}
# criando o index
client.indices.create(index = 'metacritic_games', body = index_config)

C:\Users\thiag\AppData\Local\Temp\ipykernel_17092\3893292221.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.indices.create(index = 'metacritic_games', body = index_config)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'metacritic_games'})

#### Inserindo intâncias no novo _index_
<hr>
<br>
<p style='text-align: justify'>
    Para inserir as intâncias, é feito um loop pela DataFrame, onde cada linha é salva em uma variável e então com um método do client, é instanciado dentro do index criado. Neste caso, o id está sendo passado com um iterador <i>i</i>, apenas para exemplificação.
</p>
<br>

```python
client.index(index = ..., id = ..., document = ...)
```

In [28]:
# inserindo 100 instancias no novo index
dt['release_date'] = pd.to_datetime(dt['release_date'], infer_datetime_format = True)

def insert_in_index(index_name, dt):
    i = 0;
    with tqdm(total = 100) as pbar:
        for index, row in dt.iterrows():
            body = row.to_dict()
            body['id'] = i
            client.index(index = index_name, id = i, document = body)
            i += 1
            pbar.update(1)
        
insert_in_index('metacritic_games', dt)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.35it/s]


In [29]:
# FORAM ADICIONADOS COM SUCESSO
client.count(index = 'metacritic_games')

ObjectApiResponse({'count': 100, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

#### Realizando consultas simples e atualizando dados com Elasticsearch
<hr>
<br>
<p style = 'text-align: justify'>
Para realizar uma consulta simples apenas pelo id do dado, basta usar o comando abaixo e então pegar o dicionário com as informações na key '_source'.
</p>

```python
client.get(index = ..., id = ...)
```

<p style = 'text-align: justify'>
Para realizar uma atualização de dados, basta usar o método 'update' do client, passando o id da instancia e no campo body/doc, um dicionario com apenas o campo que necessita ser atualizado.
</p>

```python
update = {'platform' : 'Playstation 5'}

client.update(index = index, id = id, doc = update)
```


In [38]:
# CONSULTANDO INSTANCIAS PELO ID
client.get(index = 'metacritic_games', id = 33)['_source']

{'metascore': 58,
 'platform': 'Wii',
 'release_date': '2010-06-21T00:00:00',
 'sort_no': 17041,
 'summary': "[WiiWare]   A crumbling mansion is home to a most peculiar picture book. Gather your courage and open the door to this odd, unsettling and slightly scary tale. Solve innovative puzzles by using the Wii Remote controller to throw animated paper bears into the picture book's unusual pages. Grab all the red candies found inside, carefully place your bears and cleverly manipulate the unique environments. Delve into the unfolding drama of Pina, the mysterious young girl residing in the mansion. Thirty exciting and challenging stages await you.",
 'title': 'The Tales of Bearsworth Manor: Puzzling Pages',
 'user_score': 'tbd',
 'id': 33}

In [40]:
client.get(index = 'metacritic_games', id = 16)['_source']

{'metascore': 77,
 'platform': 'Xbox Series X',
 'release_date': '2022-04-02T00:00:00',
 'sort_no': 7049,
 'summary': "THE FATE OF THE CITY IS IN YOUR HANDS It's been 15 years since humanity lost to the virus. The last great human settlement exists within an unforgiving, infected world, plunged into a modern dark age. During the day, bandits, factions and starving survivors roam the streets scavenging for scraps - or someone to take them from, by violence if necessary. At night the infected roam free, evacuating their dark hideouts to prey on the living. You are Aiden Caldwell, an infected survivor. Your exceptional agility and brutal combat skills make you a powerful ally and a valuable commodity in this dangerous world. You can achieve things no one else can. Enter places no one else dares. With your unique abilities you have the power to be an agent of change within this decaying metropolis.\n\nYOUR CHOICES, YOUR WORLD\nYou must make tough decisions and hard sacrifices as you decide

In [41]:
# ATUALIZANDO 2 INSTANCIAS
alteracoes = ['Playstation 5', 'Nintendo Switch']
ids = [16, 33]

for i in range(2):
    doc_update = {
        'doc': {
            'platform': alteracoes[i]
        }
    }
    client.update(index='metacritic_games', id = ids[i], body = doc_update)

C:\Users\thiag\AppData\Local\Temp\ipykernel_17092\2274960563.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  client.update(index='metacritic_games', id = ids[i], body = doc_update)


In [42]:
print(client.get(index = 'metacritic_games', id = 16)['_source'])
print()
print(client.get(index = 'metacritic_games', id = 33)['_source'])

{'metascore': 77, 'platform': 'Playstation 5', 'release_date': '2022-04-02T00:00:00', 'sort_no': 7049, 'summary': "THE FATE OF THE CITY IS IN YOUR HANDS It's been 15 years since humanity lost to the virus. The last great human settlement exists within an unforgiving, infected world, plunged into a modern dark age. During the day, bandits, factions and starving survivors roam the streets scavenging for scraps - or someone to take them from, by violence if necessary. At night the infected roam free, evacuating their dark hideouts to prey on the living. You are Aiden Caldwell, an infected survivor. Your exceptional agility and brutal combat skills make you a powerful ally and a valuable commodity in this dangerous world. You can achieve things no one else can. Enter places no one else dares. With your unique abilities you have the power to be an agent of change within this decaying metropolis.\n\nYOUR CHOICES, YOUR WORLD\nYou must make tough decisions and hard sacrifices as you decide the

#### Realizando consultas complexas com Elasticsearch
<hr>
<br>
<p style = 'text-align: justify'>
Para realizar uma consulta mais complexas no Elasticsearch (seu principal diferencial), é necesário usar o método 'search', passando a query que será a fonte da busca na base de dados.
</p>
<br>

```python
client.search(index = ..., query = {})
```
<br>

<p>
Segue uma tabela com os tipos de query que foram utilizadas no index que foi criado:
</p>
<br>

| Categoria | Tipo | Critério | Exemplo usado |
| --------- | ---- | -------- | ------------- |
| match | full-text | se a keyword estiver no texto | 'rpg' |
| range | term | documentos com o campo em um intervalo específico | { 'gte' : 20, 'lte' : 50 } |
| regexp | term | Através de expressões regulares | ma[a-z]* |

In [46]:
# REALIZANDO CONSULTAS DIFERENTES
# Pegando instancias que contem a palavra 'rpg' no campo de descrição
resp = client.search(index="metacritic_games", query={"match": {"summary" : "rpg"}})

for hit in resp['hits']['hits']:
    print(hit['_source'])

{'metascore': 72, 'platform': 'Xbox One', 'release_date': '2016-03-06T00:00:00', 'sort_no': 10449, 'summary': 'Anima Gate of Memories is a third person action RPG in which the player will explore a vast world filled with dangers and wonders. In it you take control of two characters who are trap in a war in the shadows, a secret conflict that will engulf the world into chaos.', 'title': 'Anima: Gate of Memories', 'user_score': '7.5', 'id': 40}
{'metascore': 79, 'platform': 'PC', 'release_date': '2021-10-27T00:00:00', 'sort_no': 5524, 'summary': 'The battle fire of steel ignites after 30 years\n\n"Super Robot Wars 30, a title commemorating the 30th anniversary of the series, is coming to Nintendo Switch/PlayStation®4/STEAM®.\n\nSuper Robot Wars is a simulation RPG where robots of various animations come together to fight against a common enemy.\nIn this original story, robots from various animations such as " "Mazinger Z: Infinity," "Super Electromagnetic Robot Combattler V," "Mobile Sui

In [49]:
# Pegando instancias com um metascore no intervalo [30, 50]
resp2 = client.search(index="metacritic_games", query={"range": {"metascore" :{'gte' : 30, 'lte' : 50}}})

for hit in resp2['hits']['hits']:
    print(hit['_source'])
    print()

{'metascore': 50, 'platform': 'Game Boy Advance', 'release_date': '2003-11-18T00:00:00', 'sort_no': 18666, 'summary': "You are the ultimate weapon in Terminator 3: Rise of the Machines.  Ten years after John Connor helped save mankind from annihilation, the inevitable has occurred: The mechanized Tech-Com forces have risen, and war between man and machine has begun.  As the Terminator, you must use hand-to-hand combat or your massive weaponry to protect Connor, the would-be hero of the future.  Rise of the Machines immerses you in the Terminator 3 world, weaving in and out of the movie's storyline.", 'title': 'Terminator 3: Rise of the Machines', 'user_score': 'tbd', 'id': 57}

{'metascore': 45, 'platform': 'PlayStation 2', 'release_date': '2005-01-11T00:00:00', 'sort_no': 19244, 'summary': 'In Without Warning, a radical terrorist group has targeted and seized the Peterson-Daniels Chemical Facility, a vulnerable location that jeopardizes millions of people from the devastating ecologic

In [51]:
# Pegando instancias usando expressões regulares no campo titulo
resp2 = client.search(index="metacritic_games", query={"regexp": {"title" : 'ma[a-z]*'}})

for hit in resp2['hits']['hits']:
    print(hit['_source'])
    print()

{'metascore': 73, 'platform': '3DS', 'release_date': '2016-02-12T00:00:00', 'sort_no': 9295, 'summary': "Get ready to play a near-infinite number of side-scrolling 2D Mario courses anytime, anywhere. Earn in-game medals in 100 built-in courses and access recommended courses from the Super Mario Maker game on the Wii U console. Plus, design courses and share them with other players via the StreetPass feature and local wireless. Want your cannons to shoot coins? Go for it. In this game, you call the shots, and simple touch-screen controls and interactive tutorials make designing courses a snap. Learn design tips and tricks from Mary O. and Yamamura, then share your masterpieces via the StreetPass feature and local wireless. Collaborating has never been easier now that you can share incomplete courses with other players. Finish off a course together and a special icon will commemorate the collaboration. That's one of many reasons that this is the must-play portable Mario™ game for inventi

#### Deletando instâncias pelo id e por query no Elasticsearch
<hr>
<br>
<p style = 'text-align: justify'>
Para deletar uma instância por id, basta chamar o método 'delete' e então passar as informações de index e id. Enquanto para deletar passando uma query, basta realizar a mesma operação de busca, porém com o método 'delete_by_query'
</p>
<br>

```python
client.delete(index = ..., id = ...)

client.delete_by_query(index = ..., query = {})
```

In [52]:
client.get(index = 'metacritic_games', id = 10)['_source']

{'metascore': 80,
 'platform': 'PC',
 'release_date': '2017-04-18T00:00:00',
 'sort_no': 4438,
 'summary': 'Live the life of space captain Flinthook! Armed with your mighty hookshot, your uncanny slow-motion powers and your trusty blasma pistol, plunder and fight your way through an infinite variety of randomly-assembled spaceships for treasure, loot and fame!',
 'title': 'Flinthook',
 'user_score': '7.3',
 'id': 10}

In [53]:
# REMOVENDO UMA INSTANCIA
client.delete(index = 'metacritic_games', id = 10)

ObjectApiResponse({'_index': 'metacritic_games', '_id': '10', '_version': 2, 'result': 'deleted', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 102, '_primary_term': 1})

In [55]:
client.get(index = 'metacritic_games', id = 10)

NotFoundError: NotFoundError(404, "{'_index': 'metacritic_games', '_id': '10', 'found': False}")

In [56]:
# REMOVENDO POR QUERY
client.delete_by_query(index = 'metacritic_games', query={"match": {"summary" : "rpg"}})

ObjectApiResponse({'took': 1538, 'timed_out': False, 'total': 2, 'deleted': 2, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [65]:
# REMOVENDO POR QUERY
resp3 = client.search(index = 'metacritic_games', query={"match": {"summary" : "rpg"}})
print(resp3)
for hit in resp3['hits']['hits']:
    print(hit['_source'])
    print()

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
